In [2]:
%load_ext autoreload
%autoreload 2
import netCDF4
import xarray as xr
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants, split_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate, test, calculate_global_class_weights
from hmpai.pytorch.utilities import DEVICE, set_global_seed, get_summary_str, save_model, load_model
from hmpai.pytorch.generators import SAT1Dataset, MultiXArrayDataset, MultiXArrayProbaDataset
from hmpai.data import SAT1_STAGES_ACCURACY, SAT_CLASSES_ACCURACY
from hmpai.visualization import plot_confusion_matrix
from hmpai.pytorch.normalization import *
from torchinfo import summary
from hmpai.utilities import print_results, CHANNELS_2D, AR_SAT1_CHANNELS
from torch.utils.data import DataLoader
from hmpai.pytorch.correlation import *
# from braindecode.models.eegconformer import EEGConformer
from mne.io import read_info
import os
DATA_PATH = Path(os.getenv("DATA_PATH"))
from hmpai.visualization import plot_predictions_on_epoch
from hmpai.behaviour.sat2 import read_behavioural_info

In [3]:
set_global_seed(42)
data_path_1 = DATA_PATH / "sat2/stage_data_proba_250hz_part1.nc"
data_path_2 = DATA_PATH / "sat2/stage_data_proba_250hz_part2.nc"
data_paths = [data_path_1, data_path_2]
# train_percentage=100 makes test and val 100 as well
splits = split_participants(data_paths, train_percentage=60)
labels = SAT_CLASSES_ACCURACY
whole_epoch = True
info_to_keep = ['event_name', 'participant', 'epochs']
subset_cond = None # 'speed'|'accuracy'|None
skip_samples = 0

In [4]:
norm_fn = norm_mad_zscore
statistics = {
    "global_min": -0.00014557216,
    "global_max": 0.00014740844,
    "global_mean": -2.277374212336032e-18,
    "global_std": 3.3968840765876904e-06,
    "global_median": 3.4879516e-11,
    "mad_score": 3.2237037e-06,
    "class_weights": Tensor([0.0, 0.0, 0.0, 0.0, 0.0]),
}
norm_vars = get_norm_vars_from_global_statistics(statistics, norm_fn)

train_data = MultiXArrayProbaDataset(
    data_paths,
    participants_to_keep=splits[0],
    normalization_fn=norm_fn,
    whole_epoch=whole_epoch,
    labels=labels,
    info_to_keep=info_to_keep,
    subset_cond=subset_cond,
    statistics=statistics,
    skip_samples=skip_samples,
)
class_weights = train_data.statistics["class_weights"]
test_data = MultiXArrayProbaDataset(
    data_paths,
    participants_to_keep=splits[1],
    normalization_fn=norm_fn,
    norm_vars=norm_vars,
    whole_epoch=whole_epoch,
    labels=labels,
    info_to_keep=info_to_keep,
    subset_cond=subset_cond,
    skip_samples=skip_samples,
)
val_data = MultiXArrayProbaDataset(
    data_paths,
    participants_to_keep=splits[2],
    normalization_fn=norm_fn,
    norm_vars=norm_vars,
    whole_epoch=whole_epoch,
    labels=labels,
    info_to_keep=info_to_keep,
    subset_cond=subset_cond,
    skip_samples=skip_samples,
)

In [5]:
behaviour_sat2 = read_behavioural_info(DATA_PATH / "sat2/behavioural/df_full.csv")
test_loader_sat2 = DataLoader(
    test_data, batch_size=128, shuffle=True, num_workers=0, pin_memory=True
)

In [6]:
chk_path = Path("../../models/mamba_conv.pt")
checkpoint = load_model(chk_path)

model_kwargs = {
    "embed_dim": 128,
    "n_channels": 19,
    "n_classes": len(labels),
    "n_layers": 5,
    "global_pool": False,
    "dropout": 0.1,
}
model = MambaModel(**model_kwargs)
model.load_state_dict(checkpoint["model_state_dict"])
model = model.to(DEVICE)

model.pretraining = False
model.global_pool = False
model.eval();

In [1]:
# SAT2
window_size = 0
amount_of_graphs = 5
counter = 0
n_shuffles = 5

for batch in test_loader_sat2:
    info = batch[2][0]
    lengths = torch.sum(batch[0][:, :, 0] != MASKING_VALUE, dim=1)
    for epoch, true, participant, trial, length in zip(batch[0], batch[1], info['participant'], info['epochs'], lengths):
        plot_predictions_on_epoch(
            epoch,
            true,
            SAT_CLASSES_ACCURACY,
            window_size,
            model,
            smoothing=False,
            sequence=True,
            random_perm=True,
        )
        epoch = epoch.unsqueeze(0).to(DEVICE)
        pred = model(epoch)
        # shuffled_pred_log = torch.zeros_like(pred)
        shuffled_pred = torch.zeros_like(pred)

        for _ in range(n_shuffles):
            epoch[0, :length] = epoch[0, torch.randperm(length)]
            single_shuffled_pred = model(epoch)
            # shuffled_pred_log += torch.nn.LogSoftmax(dim=2)(model(epoch))
            shuffled_pred += torch.nn.Softmax(dim=2)(model(epoch))
        # shuffled_pred_log /= n_shuffles
        shuffled_pred /= n_shuffles

        true = true[:pred.shape[1]]
        true = true + 1e-8
        true = true / true.sum(dim=-1, keepdim=True)
        true_kldiv = kldiv(pred, true)
        shuffled_kldiv = kldiv(shuffled_pred, true)

        print(f'KLDivLoss - True: {true_kldiv}, Shuffled: {shuffled_kldiv}')
        break
    break





NameError: name 'test_loader_sat2' is not defined